# Определение зон доставки курьеров на разных видах транспорта

In [ ]:
import osmnx as ox
import folium
import branca.colormap as cm
import networkx as nx
from shapely.geometry import Point, Polygon, MultiPolygon
from folium import GeoJson, GeoJsonTooltip, Choropleth
from shapely import wkt
import pandas as pd
from pandas import json_normalize
import json
import geopandas as gpd
import matplotlib.pyplot as plt
from matplotlib import cm
import seaborn as sns
# библиотеки для работы с графом OSM
from srai.regionalizers import geocode_to_region_gdf
from srai.loaders import OSMOnlineLoader, OSMNetworkType, OSMWayLoader
from srai.regionalizers import H3Regionalizer

Для анализа я выбрала холдинг Нижнего Новгорода из 10ти ресторанов - [unity restaurant group](https://urestgroup.ru/)
- [Lamvil](https://lamvil.ru/): Октябрьская 16/10
- [Scoba](https://skobar.ob-ob.ru/): Рождественская, 4
- [Мазл тоф](https://mazltov.ru/): Большая Покровская улица, 9Б
- [Маджонг](https://mahjong.ob-ob.ru/): Большая Покровская улица, 9Б
- [Три апельсина](https://restoran3apelcina.ru/): улица Нестерова, 1/9
- [Синдбад](https://sindbad.ob-ob.ru/): улица Нестерова, 1/9
- [Тихуана](https://tihuana.ru/): Большая Покровская улица, 24/22
- [Кефтэме](https://kefteme-nn.ru/): Большая Покровская улица, 24/22
- [Мама дома на Костина](https://mamadoma.ob-ob.ru/kostina-rest): Костина, 3
- [Мама дома на Горького](https://mamadomann.ru/): М.Горького 156

Для начала загрузим все точки ресторанов в один геодатафрейм и посмотрим на карте как они распределены по городу

In [2]:
lat_lamvil, lon_lamvil = 56.322531, 44.006998
lat_scoba, lon_scoba = 56.329507, 43.995036
lat_mazltov, lon_mazltov = 56.324497, 44.002725
lat_mahjong, lon_mahjong = 56.324581, 44.002744
lat_3apelcina, lon_3apelcina = 56.328903, 44.018491
lat_sindbad, lon_sindbad = 56.328948, 44.018311
lat_tihuana, lon_tihuana = 56.320857, 43.999411
lat_kefteme, lon_kefteme = 56.320651, 43.999215
lat_mama_kostina, lon_mama_kostina = 56.311931, 43.990616
lat_mama_gorkogo, lon_mama_gorkogo = 56.316918, 44.007190

In [ ]:
pt_dict = {'name': ['Lamvil', 'Scoba', 'Мазл Тоф', 'Маджонг', 'Три апельсина', 
                    'Синдбад', 'Тихуана', 'Кефтэме', 'Мама дома на Костина', 'Мама дома на Горького'], 
           'geometry': [Point(lon_lamvil, lat_lamvil), 
                        Point(lon_scoba, lat_scoba),
                        Point(lon_mazltov, lat_mazltov),
                        Point(lon_mahjong, lat_mahjong),
                        Point(lon_3apelcina, lat_3apelcina),
                        Point(lon_sindbad, lat_sindbad),
                        Point(lon_tihuana, lat_tihuana),
                        Point(lon_kefteme, lat_kefteme),
                        Point(lon_mama_kostina, lat_mama_kostina),
                        Point(lon_mama_gorkogo, lat_mama_gorkogo),]} 
df_point = gpd.GeoDataFrame(pt_dict, crs="EPSG:4326") # указываем проекцию EPSG:4326

df_point

In [ ]:
map_point = folium.Map(location=[lat_mahjong, lon_mahjong], zoom_start=14, tiles='CartoDB positron', attributionControl=0)
map_point = df_point.explore(m=map_point)
map_point

Очень интересная особенность данного холдинга в том, что шесть из десяти ресторанов находятся в одном здании и непосредственно граничат друг с другом

### Анализ конкуренции 

Посмотрим сколько конкурентов расположено рядом с выбранными нами ресторанами в пешей доступности

In [5]:
gdf = gpd.GeoDataFrame(index=[0], crs='epsg:32637', geometry=df_point.to_crs(32637).buffer(10000))
area = gdf.to_crs(4326)

In [ ]:
query = {"amenity": ['cafe', 'restaurant', 'pub']}

loader = OSMOnlineLoader()
restaurants = loader.load(area, query)

In [ ]:
# отобразим скаченные данные на карте
map = folium.Map(
    location=[area.centroid.y.iloc[0], area.centroid.x.iloc[0]], 
    zoom_start = 11,
    attributionControl=0,
    tiles='cartodbpositron'
)

# TODO
# отрисовать выгруженные объекты
folium.GeoJson(
    restaurants, control = False, 
    marker = folium.CircleMarker(
        radius = 4, weight = 0.2, 
        fill_color = 'orange', fill_opacity = 1
    )
).add_to(map)

# folium.GeoJson(...).add_to(map)

folium.GeoJson(area, style_function=lambda feature: {'fillOpacity': 0.01,'color': 'grey'}).add_to(map)

map

In [ ]:
restaurants

### Расчет изохрон

- **Пешая**: радиус времени 10-15 минут.

$ Расстояние = Скорость \times Время $

где:
- Средняя скорость пешехода в м/с = 1.39 м/с (~5 км/ч),
- Время = $ 15 \times 60 $ = 900 секунд. (для расчетов переводим минуты в секунды)

Подставляем значения для пешеходного радиуса в метрах:

$ Расстояние = 1.39 \, \text{м/с} \times 900 \, \text{секунд} = 1250 \, \text{метров} $


In [9]:
walking_speed = 1.39 
time_in_seconds = 15 * 60 
walk_radius = walking_speed * time_in_seconds

In [10]:
point = df_point[df_point['name'] == 'Тихуана']['geometry'].geometry.iloc[0]

lon, lat = point.x, point.y

center_point=(lat, lon)

Создаем изохроны для пешей доступности всех десяти ресторанов

In [ ]:
isochrones = []

for _, row in df_point.iterrows():

    lat, lon = row['geometry'].y, row['geometry'].x  # Координаты ресторана берем из созданного выше геодатафрейма
    
    # Строим пешеходнй граф
    walk_graph = ox.graph_from_point((lat, lon), dist=walk_radius, dist_type="network", network_type="walk", simplify=True)
    
    # Находим ближайший узел
    center_node = ox.nearest_nodes(walk_graph, X=lon, Y=lat)
    
    # Рассчитываем длину от него до всех узлов
    lengths = nx.single_source_dijkstra_path_length(walk_graph, center_node, weight="length")
    
    # Узлы должны быть в пределах изохроны
    reachable_nodes = [node for node, dist in lengths.items() if dist <= walk_radius]
    subgraph = walk_graph.subgraph(reachable_nodes)
    
    # Преобразуем узлы в полигоны (Convex Hull)
    node_points = [Point(data['x'], data['y']) for node, data in subgraph.nodes(data=True)]
    isochrone_polygon = gpd.GeoSeries(node_points).unary_union.convex_hull
    
    # Добавляем в список
    isochrones.append({'name': row['name'], 'geometry': isochrone_polygon})

isochrones_walk_gdf = gpd.GeoDataFrame(isochrones, crs="EPSG:4326")

In [ ]:
isochrones_walk_gdf

Создаем визуализацию изохрон на карте с помощью folium, причем для каждого ресторана создаем отдельный слой

In [ ]:
m = folium.Map(location=[df_point['geometry'].y.mean(), df_point['geometry'].x.mean()], zoom_start=13, tiles="cartodbpositron")

for _, row in isochrones_walk_gdf.iterrows():
    fg = folium.FeatureGroup(name=row['name'])

    folium.GeoJson(
        row['geometry'],
        style_function=lambda x: {'color': 'green', 'fillOpacity': 0.3, 'weight': 2},
        name=row['name']
    ).add_to(fg)

    restaurant = df_point.loc[df_point['name'] == row['name']].iloc[0]

    folium.Marker(
        [restaurant['geometry'].y, restaurant['geometry'].x],
        popup=restaurant['name'],
        icon=folium.Icon(color="black", icon="cutlery")
    ).add_to(fg)

    fg.add_to(m)

# Добавляем панель управления слоями
folium.LayerControl(collapsed=False).add_to(m)

m

Благодаря изохронам мы можем быстро посчитать сколько конкурентов располагается в пешей доступности от ресторанов

In [ ]:
isochrones_walk_gdf['restaurants_within'] = 0

# Для каждой изохроны считаем, сколько точек попадает внутрь
for idx, isochrone in isochrones_walk_gdf.iterrows():
    # Считаем количество точек ресторанов, попадающих внутрь полигона изохроны
    count = restaurants.within(isochrone.geometry).sum()
    # Записываем результат в соответствующую строку
    isochrones_walk_gdf.at[idx, 'restaurants_within'] = count

isochrones_walk_gdf[['name', 'restaurants_within']]

Получается, что больше всех конкурентов у Тихуаны, это оправдано, так как она находится в середине самой туристической улицы Нижнего, дальше идут Мазл Тоф и Маджонг, потому что они так же находятся на этой улице.

Сделаем расчеты скорости велосипеда и автомобиля из км/ч в м/с

In [ ]:
average_speed_bicycle_kmh = 15  # велосипед
average_speed_car_kmh = 45     # автомобиль

# Перевод в м/с (1 км/ч = 1000 м / 3600 с)
average_speed_bicycle_ms = average_speed_bicycle_kmh * 1000 / 3600
average_speed_car_ms = average_speed_car_kmh * 1000 / 3600

average_speed_bicycle_ms, average_speed_car_ms

- **Велосипедная**: радиус 15-30 минут.

$ Расстояние = Скорость \times Время $

где:
- Средняя скорость велосипеда в м/с = 4.167 м/с (~15 км/ч),
- Время = $ 22.5 \times 60 $ = 1350 секунд.

Подставляем значения для пешеходного радиуса:

$ Расстояние = 4.167 \, \text{м/с} \times 1350 \, \text{секунд} = 5\ 625.45 \, \text{метров} $

In [ ]:
time_in_seconds = 22.5 * 60  # 22 с половиной минуты в секундах
bike_radius = average_speed_bicycle_ms * time_in_seconds  # радиус в метрах
bike_radius

In [ ]:
isochrones_bike = []

for _, row in df_point.iterrows():

    lat, lon = row['geometry'].y, row['geometry'].x  

    bike_graph = ox.graph_from_point((lat, lon), dist=bike_radius, dist_type="network", network_type="bike", simplify=True)
    
    center_node = ox.nearest_nodes(bike_graph, X=lon, Y=lat)
    
    lengths = nx.single_source_dijkstra_path_length(bike_graph, center_node, weight="length")
    
    reachable_nodes = [node for node, dist in lengths.items() if dist <= bike_radius]
    subgraph = bike_graph.subgraph(reachable_nodes)
    
    node_points = [Point(data['x'], data['y']) for node, data in subgraph.nodes(data=True)]
    isochrone_polygon = gpd.GeoSeries(node_points).unary_union.convex_hull
    
    isochrones_bike.append({'name': row['name'], 'geometry': isochrone_polygon})

isochrones_bike_gdf = gpd.GeoDataFrame(isochrones_bike, crs="EPSG:4326")

In [ ]:
m = folium.Map(location=[df_point['geometry'].y.mean(), df_point['geometry'].x.mean()], zoom_start=12, tiles="cartodbpositron")

for _, row in isochrones_bike_gdf.iterrows():

    fg = folium.FeatureGroup(name=row['name'])
    
    folium.GeoJson(
        row['geometry'],
        style_function=lambda x: {'color': 'orange', 'fillOpacity': 0.2, 'weight': 2},
        name=row['name']
    ).add_to(fg)

    restaurant = df_point.loc[df_point['name'] == row['name']].iloc[0]
    
    folium.Marker(
        [restaurant['geometry'].y, restaurant['geometry'].x],
        popup=restaurant['name'],
        icon=folium.Icon(color="black", icon="cutlery")
    ).add_to(fg)
    
    fg.add_to(m)

folium.LayerControl(collapsed=False).add_to(m)

m

- **Автомобильная**: радиус 30-60 минут.

$ Расстояние = Скорость \times Время $

где:
- Средняя скорость автомобиля в черте города в м/с = 12.5 м/с (~45 км/ч),
- Время = $ 45 \times 60 $ = 2700 секунд.

Подставляем значения для пешеходного радиуса:

$ Расстояние = 12.5 \, \text{м/с} \times 2700 \, \text{секунд} = 33\ 750 \, \text{метров} $

In [ ]:
time_in_seconds = 45 * 60  # 45 минут переводим в секунды
drive_radius = average_speed_car_ms * time_in_seconds  # радиус в метрах
drive_radius

In [ ]:
isochrones_drive = []

for _, row in df_point.iterrows():

    lat, lon = row['geometry'].y, row['geometry'].x  

    drive_graph = ox.graph_from_point((lat, lon), dist=drive_radius, dist_type="network", network_type="drive", simplify=True)

    center_node = ox.nearest_nodes(drive_graph, X=lon, Y=lat)
   
    lengths = nx.single_source_dijkstra_path_length(drive_graph, center_node, weight="length")
    
    reachable_nodes = [node for node, dist in lengths.items() if dist <= drive_radius]
    subgraph = drive_graph.subgraph(reachable_nodes)
    
    node_points = [Point(data['x'], data['y']) for node, data in subgraph.nodes(data=True)]
    isochrone_polygon = gpd.GeoSeries(node_points).unary_union.convex_hull
    
    isochrones_drive.append({'name': row['name'], 'geometry': isochrone_polygon})

isochrones_drive_gdf = gpd.GeoDataFrame(isochrones_drive, crs="EPSG:4326")

In [ ]:
m = folium.Map(location=[df_point['geometry'].y.mean(), df_point['geometry'].x.mean()], zoom_start=10, tiles="cartodbpositron")

for _, row in isochrones_drive_gdf.iterrows():

    fg = folium.FeatureGroup(name=row['name'])
    
    folium.GeoJson(
        row['geometry'],
        style_function=lambda x: {'color': 'purple', 'fillOpacity': 0.2, 'weight': 2},
        name=row['name']
    ).add_to(fg)
    
    restaurant = df_point.loc[df_point['name'] == row['name']].iloc[0]
    
    folium.Marker(
        [restaurant['geometry'].y, restaurant['geometry'].x],
        popup=restaurant['name'],
        icon=folium.Icon(color="red", icon="cutlery")
    ).add_to(fg)
    
    fg.add_to(m)

folium.LayerControl(collapsed=False).add_to(m)

m

In [ ]:
isochrones_drive_gdf

Визуализируем для каждой точки все виды изохрон

In [ ]:
m = folium.Map(location=[df_point['geometry'].y.mean(), df_point['geometry'].x.mean()], zoom_start=10, tiles="cartodbpositron")

# Цвета для типов изохрон оставляем прежними, чтобы было легче ориентироваться
isochrone_colors = {
    "walk": "green",
    "bike": "orange",
    "drive": "purple"
}
# Указываем ресторан, который будет виден при загрузке карты, чтобы не перегружать визуализацию
default_restaurant = "Lamvil"

# Добавляем слои для каждого ресторана
for _, restaurant in df_point.iterrows():
    restaurant_name = restaurant['name']
    restaurant_lat = restaurant['geometry'].y
    restaurant_lon = restaurant['geometry'].x
    
    # Параметр `show` определяет, виден ли слой при загрузке
    show_layer = (restaurant_name == default_restaurant)

    restaurant_group = folium.FeatureGroup(name=f"{restaurant_name}", show=show_layer)
    
    # Добавляем изохроны разных типов для ресторана
    for delivery_type, color in isochrone_colors.items():
        # Фильтруем данные для текущего ресторана и типа доставки
        isochrone_gdf = globals()[f"isochrones_{delivery_type}_gdf"]
        isochrone = isochrone_gdf[isochrone_gdf['name'] == restaurant_name]
        
        if not isochrone.empty:
            folium.GeoJson(
                isochrone.iloc[0]['geometry'],
                style_function=lambda x, color=color: {'color': color, 'fillOpacity': 0.15, 'weight': 2},
                name=f"{restaurant_name} ({delivery_type})"
            ).add_to(restaurant_group)
    
    folium.Marker(
        [restaurant_lat, restaurant_lon],
        popup=restaurant_name,
        icon=folium.Icon(color="red", icon="cutlery")
    ).add_to(restaurant_group)
    
    restaurant_group.add_to(m)

folium.LayerControl(collapsed=False).add_to(m)

m

Зоны автомобильной доставки для всех ресторанов на карте практически не отличаются и затрагивают не только сам Нижний Новгород, но и пригород.

### Анализ потенциала зоны доставки

Во-первых, посмотрим сколько человек, в среднем, проживает на территории, которую затрагивает доставка. Сколько человек, проживающих на затронутой територии, попадает в зону доставки.

Во-вторых, проанализируем сколько домов и коммерческих зданий(офисов) попало в изохрону ресторана, чтобы определить потенциал полученной зоны доставки за счет них.

# 1. 

Загружаем данные о проживающих в Нижнем Новгороде

In [24]:
popular = gpd.read_file('kontur_population_RU_20231101.gpkg')

In [ ]:
popular

In [26]:
# Преобразуйте CRS в EPSG:4326 для работы с folium
popular = popular.to_crs(epsg=4326)

In [27]:
center_point = Point(center_point)

In [ ]:
isochrones_walk_gdf

Сделаем пространственное соединение между гексагонами и изохронами

1. Пешие изохроны

In [ ]:
points_walk_osm = popular.sjoin(isochrones_walk_gdf, how='inner', predicate='intersects')

points_walk_osm

In [ ]:
walk_population_summary = points_walk_osm.groupby('name')['population'].sum().reset_index()

walk_population_summary

In [ ]:
sns.set(style="whitegrid")

plt.figure(figsize=(10, 6))
bar_plot = sns.barplot(x='name', y='population', data=walk_population_summary, palette='viridis')

for index, value in enumerate(walk_population_summary['population']):
    plt.text(index, value + 1000, str(value)[:5], ha='center', va='bottom', fontsize=10)

plt.title('Количество человек, попавших в изохрону пешей доступности доставки ресторана', fontsize=16)
plt.xlabel('Location', fontsize=12)
plt.ylabel('Population', fontsize=12)
plt.xticks(rotation=45, ha="right")
plt.tight_layout()

plt.show()

Рассчитаем плотность ресторанов в пешей доступности к населению, т.е. сколько ресторанов приходится на 1000 человек - это **демографический** показатель. 

In [32]:
data = isochrones_walk_gdf.merge(walk_population_summary, on='name')

In [ ]:
data["restaurants_per_1000"] = data["restaurants_within"] / data["population"] * 1000
data

In [ ]:
fig, ax = plt.subplots(figsize=(10, 10))
data.plot(column='restaurants_per_1000', ax=ax, legend=True, cmap='OrRd', legend_kwds={'label': "Ресторанов на 1000 человек"})
data.boundary.plot(ax=ax, linewidth=1, color='black')
plt.title('Плотность ресторанов на 1000 человек по зонам доставки')
plt.show()

А еще можно определить **географический** показатель - количество ресторанов на единицу площади ($км^2$). Для этого сначала необходимо рассчитать площади изохрон.

In [77]:
data.set_crs(epsg=4326, inplace=True)

data = data.to_crs(epsg=32637)

In [ ]:
data['area_km2'] = data.geometry.area/1000000 # Делим на миллион чтобы перевести квадратные метры в километры

data

In [ ]:
data['restaurant_density'] = data['restaurants_within']/data['area_km2']
data

In [ ]:
print(data.crs)  # Должно быть EPSG:4326 (WGS84)
if data.crs != 'EPSG:4326':
    data = data.to_crs(epsg=4326)  # Преобразуем в WGS84

In [ ]:
center_point = [data.geometry.centroid.y.mean(), data.geometry.centroid.x.mean()]
m = folium.Map(location=center_point, zoom_start=14, tiles="cartodbpositron")

# Создаем цветовой градиент OrRd
colormap = cm.LinearColormap(
    colors=['white', 'orange', 'red'],  # OrRd-like colors
    vmin=data['restaurant_density'].min(),
    vmax=data['restaurant_density'].max()
)

# Добавляем градиент в легенду
colormap.caption = 'Плотность ресторанов в расчете на 1000 человек'
m.add_child(colormap)

# Создаем отдельный слой для каждой изохроны
for idx, row in data.iterrows():
    # Создаем слой для текущей изохроны
    layer = folium.FeatureGroup(name=row['name'])

    # Добавляем изохрону на слой
    folium.GeoJson(
        row['geometry'],
        style_function=lambda x, row=row: {
            'fillColor': colormap(row['restaurant_density']),
            'color': 'black',
            'weight': 1,
            'fillOpacity': 0.6
        },
        tooltip=f"Ресторанов на 1000 человек: {row['restaurant_density']:.2f}<br>Площадь: {row['area_km2']:.2f} км²"
    ).add_to(layer)

    # Добавляем слой на карту
    layer.add_to(m)

# Добавляем управление слоями
folium.LayerControl().add_to(m)
m

2. Велосипедная доставка

In [ ]:
points_bike_osm = popular.sjoin(isochrones_bike_gdf, how='inner', predicate='intersects')

points_bike_osm

In [ ]:
bike_population_summary = points_bike_osm.groupby('name')['population'].sum().reset_index()

bike_population_summary

In [ ]:
sns.set(style="whitegrid")

plt.figure(figsize=(10, 6))
bar_plot = sns.barplot(x='name', y='population', data=bike_population_summary, palette='viridis')

for index, value in enumerate(bike_population_summary['population']):
    plt.text(index, value + 1000, str(value)[:6], ha='center', va='bottom', fontsize=10)

plt.title('Количество человек, попавших в изохрону велосипедной доступности доставки ресторана', fontsize=16)
plt.xlabel('Location', fontsize=12)
plt.ylabel('Population', fontsize=12)
plt.xticks(rotation=45, ha="right")
plt.tight_layout()

plt.show()

3. Автомобильная доставка

In [ ]:
points_drive_osm = popular.sjoin(isochrones_drive_gdf, how='inner', predicate='intersects')

points_drive_osm

In [ ]:
drive_population_summary = points_drive_osm.groupby('name')['population'].sum().reset_index()

drive_population_summary

In [ ]:
sns.set(style="whitegrid")

plt.figure(figsize=(10, 6))
bar_plot = sns.barplot(x='name', y='population', data=drive_population_summary, palette='viridis')
for index, value in enumerate(drive_population_summary['population']):
    plt.text(index, value + 1000, str(value)[:7], ha='center', va='bottom', fontsize=10)

plt.title('Количество человек, попавших в изохрону автомобильной доступности доставки ресторана', fontsize=16)
plt.xlabel('Location', fontsize=12)
plt.ylabel('Population', fontsize=12)
plt.xticks(rotation=45, ha="right")
plt.tight_layout()

plt.show()

Сделаем единую табличку

In [41]:
combined_summary = pd.DataFrame()

Объединяем все получившиеся датафреймы в один по столбцу *name*

In [ ]:
combined_summary = walk_population_summary.merge(bike_population_summary, on='name', suffixes=('_walk', '_bike'))
combined_summary = combined_summary.merge(drive_population_summary, on='name', suffixes=('', '_drive'))

combined_summary

Сделает круговые диаграммы для визуазации для каждого ресторана

In [ ]:
fig, axes = plt.subplots(2, 5, figsize=(15, 6))
axes = axes.flatten()

# Цвета для круговой диаграммы
colors = ['#ffb300', '#00945b', '#a3339e']

# Проходим по каждому ресторану
for i, ax in enumerate(axes):
    if i < len(combined_summary["name"]):
        # Данные для текущего ресторана
        restaurant_name = combined_summary["name"][i]
        walk = combined_summary["population_walk"][i]
        bike = combined_summary["population_bike"][i]
        total = combined_summary["population"][i] - walk - bike
        
        # Данные для круговой диаграммы
        sizes = [walk, bike, total]
        labels = ['Пешком', 'Велосипед', 'Автомобиль']
        
        # Круговая диаграмма
        ax.pie(sizes, labels=labels, colors=colors, autopct='%1.1f%%', startangle=90, textprops={'fontsize': 10})
        ax.set_title(restaurant_name, fontsize=15)
    #else:
        # Если ресторанов меньше, чем subplot-ов, убираем лишние
        #ax.axis('off')

fig.suptitle("Распределение роличества населения в зависимости от типа доставки", fontsize=16)

# Убираем лишнее пространство между диаграммами
plt.tight_layout(rect=[0, 0, 1, 0.95])

plt.show()

Тепловая карта по количеству людей

In [ ]:
plt.figure(figsize=(12, 6))
sns.heatmap(combined_summary.set_index('name')[['population_walk', 'population_bike', 'population']], 
            annot=True, fmt=".0f", cmap="YlGnBu")
plt.title('Количество людей, попавших в доставку', fontsize=16)
plt.xlabel('Метод доставки', fontsize=14)
plt.ylabel('Рестораны', fontsize=14)
plt.show()

Так же интересно посмотреть распределение по районам

Для этого сначала выгружаем данные о районам Нижнего, все что в них не попадает будем считать пригородом

In [45]:
with open('updated_only_NN.geojson', 'r', encoding='utf-8') as f:
        NN_data = json.load(f)

In [ ]:
regions_NN = gpd.GeoDataFrame(NN_data)
regions_NN

In [ ]:
merged_data = json_normalize(regions_NN['features'])
merged_data = merged_data.drop(columns=['geometry.type', 'type'])

# Преобразование координат в объекты MultiPolygon
merged_data['geometry.coordinates'] = merged_data['geometry.coordinates'].apply(lambda coords: MultiPolygon(coords))

# Создание GeoDataFrame
geo_data = gpd.GeoDataFrame(merged_data, geometry='geometry.coordinates')
geo_data

Сначала посмотрим сколько человек проживает в районам Нижнего

In [ ]:
district_population = {}

for _, area in geo_data.iterrows():
    area_name = area['properties.local_name']
    area_geometry = area['geometry.coordinates']

    # Находим пересекающиеся с районом полигоны населения
    intersecting_population = popular[popular.geometry.intersects(area_geometry)]

    # Суммируем количество людей в пересекающихся областях
    total_population = intersecting_population['population'].sum()

    district_population[area_name] = total_population

# Создаём DataFrame с результатами
district_population_df = pd.DataFrame(list(district_population.items()), columns=['district', 'population'])

district_population_df

In [49]:
district_population_df = district_population_df.rename(columns={'population': 'population_all'})

Считаем сколько человек попадает в доставку по районам

In [ ]:
delivery_population_by_district = []

# Функция для обработки каждого типа изохроны
def calculate_population_in_isochrone(isochrones_gdf, transport_type):
    for _, restaurant in isochrones_gdf.iterrows():
        restaurant_name = restaurant['name']
        restaurant_geometry = restaurant['geometry']
        
        # Для каждого ресторана, находим пересекающиеся районы
        for _, area in geo_data.iterrows():
            area_name = area['properties.local_name']
            area_geometry = area['geometry.coordinates']

            # Находим пересечение изохроны ресторана с районом
            intersection = restaurant_geometry.intersection(area_geometry)

            # Если есть пересечение, вычисляем площадь пересечения
            if intersection.is_valid and not intersection.is_empty:
                intersection_area = intersection.area
                area_total = area_geometry.area

                # Вычисляем пропорцию пересечения по площади
                intersection_ratio = intersection_area / area_total

                # Находим население района
                district_population = popular[popular.geometry.intersects(area_geometry)]['population'].sum()

                # Вычисляем, сколько людей попадает в изохрону, пропорционально площади
                population_in_isochrone = district_population * intersection_ratio

                # Добавляем данные в итоговый список
                delivery_population_by_district.append({
                    'restaurant': restaurant_name,
                    'isochrone_type': transport_type,
                    'district': area_name,
                    'population': population_in_isochrone
                })

# Обрабатываем каждый тип изохроны
calculate_population_in_isochrone(isochrones_walk_gdf, 'walk')
calculate_population_in_isochrone(isochrones_bike_gdf, 'bike')
calculate_population_in_isochrone(isochrones_drive_gdf, 'drive')

# Преобразуем результат в DataFrame
delivery_population_df = pd.DataFrame(delivery_population_by_district)

delivery_population_df

In [51]:
geo_data = geo_data.set_index('id').join(district_population_df.set_index('district'))

In [ ]:
geo_data

Визуализируем границы районов на карте и точки наших ресторанов, чтобы понимать в каком территориано районе находится ресторан

In [ ]:
center_point = [geo_data.geometry.centroid.y.mean(), geo_data.geometry.centroid.x.mean()]
m = folium.Map(location=center_point, zoom_start=11, tiles="CartoDB Positron")

# Добавляем Choropleth слой
choropleth = folium.Choropleth(
    geo_data=geo_data.to_json(),  # Преобразуем GeoDataFrame в GeoJSON
    name='choropleth',
    data=geo_data,
    columns=['properties.local_name', 'population_all'],
    key_on='id',  # Связь между geo_data и данными по районам
    fill_color='YlGnBu',  # Цветовая схема
    fill_opacity=0.6,
    line_opacity=0.2,
    legend_name='Population',
).add_to(m)

choropleth.geojson.add_child(
            folium.features.GeoJsonTooltip(fields=['properties.local_name', 'population_all'], aliases=['Район', 'Население'])
        )

m = df_point.explore(m=m)

# Добавляем контроль слоев (для скрытия/отображения районов на карте)
folium.LayerControl().add_to(m)

m

Теперь посмотрим какой процент населения района попадает в зону доставки

In [54]:
# Добавляем столбец с процентами от населения
delivery_population_df['percentage'] = delivery_population_df['population'] / delivery_population_df.groupby('district')['population'].transform('sum') * 100

In [ ]:
heatmap_data = delivery_population_df.pivot_table(index='district', columns='isochrone_type', values='percentage', aggfunc='sum')

# Строим тепловую карту
plt.figure(figsize=(12, 8))
sns.heatmap(heatmap_data, annot=True, cmap='YlGnBu', cbar_kws={'label': 'Процент людей'}, fmt='.1f')

# Настроим график
plt.title('Процент людей в доставке по типам изохрон и районам')
plt.ylabel('Районы')
plt.xlabel('Типы изохрон')

# Покажем график
plt.tight_layout()
plt.show()

In [ ]:
# Группируем данные по району и типу изохрона
district_aggregated = delivery_population_df.groupby(['district', 'isochrone_type'])['population'].sum().unstack().fillna(0)

colors = ['#ffb300', '#00945b', '#a3339e']

# Настроим количество строк и столбцов для подграфиков
n_rows = len(district_aggregated) // 3 + (len(district_aggregated) % 3 > 0)  # Считаем количество строк для подграфиков
n_cols = 3  # Ставим 3 столбца

# Создаем фигуру с подграфиками
fig, axes = plt.subplots(n_rows, n_cols, figsize=(15, n_rows * 5))

# Плоский список осей
axes = axes.flatten()

# Строим круговую диаграмму для каждого района
for i, district in enumerate(district_aggregated.index):
    sizes = district_aggregated.loc[district]
    
    # Убираем типы с нулевым значением
    sizes = sizes[sizes > 0]
    labels = sizes.index
    
    # Строим круговую диаграмму на соответствующем подграфике
    axes[i].pie(sizes, labels=labels, autopct='%1.1f%%', startangle=90, 
                colors=colors, wedgeprops={'edgecolor': 'black'})
    axes[i].set_title(f"Процент людей в доставке для {district}")

# Оставляем пустые подграфики, если они есть
for j in range(i + 1, len(axes)):
    axes[j].axis('off')

# Подгоняем элементы
plt.tight_layout()
plt.show()

# 2.

Выгрузаем все данные по зданиям в Нижегородской области из OSM

In [ ]:
area = geocode_to_region_gdf("Нижегородская область")
                             
# Создаем объект для загрузки данных из OpenStreetMap
loader = OSMOnlineLoader()

# Выполняем запрос для зданий (фильтруем по тегу 'building')
osm_buildings_data = loader.load(area, tags={'building': True})


In [ ]:
osm_buildings_data.sample(10)

In [ ]:
osm_buildings_data.info()

In [ ]:
osm_buildings_data['building'].unique()

Нам нужно выбрать необходимые для доставок типы зданий:

*Жилые здания:* residential, apartments, house, dormitory, semidetached_house, bungalow, detached, terrace.

*Офисы и коммерческие здания:* office, commercial, retail.

*Склады и производственные здания:* warehouse, industrial, manufacture, storage_tank, depot.

*Учреждения, которые могут заказывать доставку:* school, hospital, kindergarten, university, college, clinic.

*Общественные здания и точки сбора людей:* community_centre, stadium, sports_centre, sports_hall, public_building, townhall, train_station.

*Отели:* hotel.

*Религиозные здания:* Если есть заказы на мероприятия — church, mosque, synagogue, cathedral, temple.

Еще я выбрала *yes* как тип зданий, так как в него входят много зданий (в основном жилые дома/жилые многоэтажки)

In [61]:
useful_building_types = [
    'residential', 'apartments', 'house', 'dormitory', 'semidetached_house', 
    'bungalow', 'detached', 'terrace', 'office', 'commercial', 'retail', 
    'warehouse', 'industrial', 'manufacture', 'storage_tank', 'depot', 'school', 
    'hospital', 'kindergarten', 'university', 'college', 'clinic', 
    'community_centre', 'stadium', 'sports_centre', 'sports_hall', 
    'public_building', 'townhall', 'train_station', 'hotel', 'church', 
    'mosque', 'synagogue', 'cathedral', 'temple', 'yes'
]

filtered_osm_data = osm_buildings_data[
    osm_buildings_data['building'].isin(useful_building_types)
]


In [ ]:
filtered_osm_data

Теперь выделим только здания, которые попадают в изохроны

- в пешие изохроны

In [ ]:
# Проверяем, чтобы системы координат совпадали
filtered_buildings = filtered_osm_data.to_crs(isochrones_walk_gdf.crs)

# Пространственное соединение зданий и районов
filtered_buildings = gpd.sjoin(filtered_buildings, geo_data, how="inner", op="within")

filtered_buildings

In [ ]:
filtered_buildings.reset_index(drop=True, inplace=True)
filtered_buildings

In [65]:
# Переименовываем колонки, если они есть
if 'index_left' in filtered_buildings.columns:
    filtered_buildings = filtered_buildings.rename(columns={'index_left': 'index_left_old'})
if 'index_right' in filtered_buildings.columns:
    filtered_buildings = filtered_buildings.rename(columns={'index_right': 'index_right_old'})

In [ ]:
buildings_in_walk_isochrones = gpd.sjoin(
    filtered_buildings, 
    isochrones_walk_gdf, 
    how="inner",  # Оставляем только здания, которые попадают в изохроны
    op="within"   # Здания должны находиться внутри изохрон
)

# Убираем дубликаты (если здание попадает в несколько изохрон)
buildings_in_walk_isochrones = buildings_in_walk_isochrones.drop_duplicates(subset=['geometry'])

buildings_in_walk_isochrones = buildings_in_walk_isochrones.drop(columns=['index_right'])

# Сбрасываем индекс
buildings_in_walk_isochrones.reset_index(drop=True, inplace=True)

buildings_in_walk_isochrones

In [ ]:
building_walk_counts_df = buildings_in_walk_isochrones.groupby('building').size().reset_index(name='count')
building_walk_counts_df

In [ ]:
building_type_counts_with_district = buildings_in_walk_isochrones.groupby(
    ['name', 'properties.local_name', 'building']
).size().reset_index(name='count')

# Преобразуем в таблицу
building_type_pivot_with_district = building_type_counts_with_district.pivot(
    index=['name', 'properties.local_name'], columns='building', values='count'
).fillna(0)

building_type_pivot_with_district

In [ ]:
district_walk_data = building_type_pivot_with_district.reset_index().melt(
    id_vars=['name', 'properties.local_name'], 
    var_name='building_type', 
    value_name='count'
)

district_walk_data

In [ ]:
# Переименуем колонку 'yes' в 'дома'
table = building_type_pivot_with_district.rename(columns={'yes': 'дома'})

# Создаем новую колонку 'другие здания', суммируя все остальные колонки
table['другие здания'] = table.drop(columns=['дома']).sum(axis=1)

# Оставляем только нужные колонки
table_simplified = table[['дома', 'другие здания']]

table_simplified

In [ ]:
# Преобразуем DataFrame в "длинный" формат
table_long = table_simplified.reset_index().melt(
    id_vars=['name', 'properties.local_name'], 
    var_name='building_type', 
    value_name='count'
)

# Визуализация столбчатой диаграммы с накоплением
plt.figure(figsize=(14, 8))
sns.barplot(data=table_long, x='name', y='count', hue='building_type')
plt.title('Количество домов и других зданий для каждого ресторана и района')
plt.xlabel('Ресторан и район')
plt.ylabel('Количество зданий')
plt.legend(title='Тип здания', bbox_to_anchor=(1.05, 1), loc='upper left')
plt.xticks(rotation=45)
plt.show()

In [64]:
center_point=(lat, lon)

In [ ]:
map = folium.Map(location=center_point, zoom_start=11, attributionControl=0, tiles='cartodbpositron')

default_restaurant = "Lamvil"  # Укажите ресторан, отображаемый по умолчанию

# Создаём слои для каждого ресторана
for _, restaurant in df_point.iterrows():
    restaurant_name = restaurant['name']
    restaurant_lat = restaurant['geometry'].y
    restaurant_lon = restaurant['geometry'].x

    # Показывать слой по умолчанию
    show_layer = (restaurant_name == default_restaurant)

    # Создаем группу для каждого ресторана
    restaurant_group = folium.FeatureGroup(name=restaurant_name, show=show_layer)

    # Изохрона ресторана
    isochron = isochrones_walk_gdf[isochrones_walk_gdf['name'] == restaurant_name].iloc[0]

    folium.GeoJson(
        isochron['geometry'],
        style_function=lambda x: {'color': 'green', 'fillOpacity': 0.0, 'weight': 2},
    ).add_to(restaurant_group)

    # Здания, попадающие в изохрону
    filtered_buildings = buildings_in_walk_isochrones[
        buildings_in_walk_isochrones.intersects(isochron['geometry'])
    ]

    # Убедитесь, что остаются только полигоны
    filtered_buildings = filtered_buildings[filtered_buildings.geometry.type == 'Polygon']

    if not filtered_buildings.empty:
        folium.GeoJson(
            filtered_buildings,
            style_function=lambda feature: {
                'fillColor': 'green',
                'color': 'green',
                'weight': 0.2,
                'fillOpacity': 0.6
            },
            highlight_function=lambda feature: {
                'fillColor': 'red',
                'color': 'red',
                'weight': 1,
                'fillOpacity': 0.9
            },
        ).add_to(restaurant_group)

    # Добавляем метку ресторана
    folium.Marker(
        [restaurant_lat, restaurant_lon],
        popup=restaurant_name,
        icon=folium.Icon(color="black", icon="cutlery")
    ).add_to(restaurant_group)

    restaurant_group.add_to(map)

# Панель управления
folium.LayerControl(collapsed=False).add_to(map)

map

- велосипедные доставки

In [ ]:
filtered_buildings = filtered_osm_data.to_crs(isochrones_bike_gdf.crs)
# Пространственное соединение зданий и районов
filtered_buildings = gpd.sjoin(filtered_buildings, geo_data, how="inner", op="within")
filtered_buildings.reset_index(drop=True, inplace=True)

filtered_buildings

In [112]:
# Переименовываем колонки, если они есть
if 'index_left' in filtered_buildings.columns:
    filtered_buildings = filtered_buildings.rename(columns={'index_left': 'index_left_old'})
if 'index_right' in filtered_buildings.columns:
    filtered_buildings = filtered_buildings.rename(columns={'index_right': 'index_right_old'})

In [ ]:
buildings_in_bike_isochrones = gpd.sjoin(
    filtered_buildings, 
    isochrones_bike_gdf, 
    how="inner",  # Оставляем только здания, которые попадают в изохроны
    op="within"   # Здания должны находиться внутри изохрон
)

# Убираем дубликаты (если здание попадает в несколько изохрон)
buildings_in_bike_isochrones = buildings_in_bike_isochrones.drop_duplicates(subset=['geometry'])

# Удаляем лишние колонки (если нужно)
buildings_in_bike_isochrones = buildings_in_bike_isochrones.drop(columns=['index_right'])

# Сбрасываем индекс
buildings_in_bike_isochrones.reset_index(drop=True, inplace=True)

buildings_in_bike_isochrones

In [ ]:
building_bike_counts_df = buildings_in_bike_isochrones.groupby('building').size().reset_index(name='count')
building_bike_counts_df = building_bike_counts_df.replace({'building': {'yes': 'living houses'}}).sort_values(by='building')

building_bike_counts_df

In [ ]:
# Группируем данные в новые категории
def categorize_building(building_type):
    if building_type in ['apartments', 'detached', 'house', 'residential', 'living houses']:
        return 'Жилые дома'
    elif building_type in ['commercial', 'office', 'retail', 'hotel', 'warehouse', 'industrial']:
        return 'Коммерческие здания'
    elif building_type in ['college', 'school', 'university', 'kindergarten', 'dormitory']:
        return 'Учебные заведения'
    elif building_type in ['hospital', 'stadium', 'synagogue', 'cathedral', 'church']:
        return 'Другое'
    else:
        return 'Другое'

# Применяем группировку
building_bike_counts_df['category'] = building_bike_counts_df['building'].apply(categorize_building)

# Группируем данные по новой категории и суммируем количество
grouped_data = building_bike_counts_df.groupby('category')['count'].sum().reset_index()

grouped_data

In [ ]:
# Создаем цветовую палитру
colors = ['purple', 'skyblue', 'gold', 'orange']
explode = (0, 0.3, 0.2, 0.1)  # Выделяем маленькие кусочки
# Визуализация круговой диаграммы
fig, ax = plt.subplots(figsize=(7, 6))
wedges, texts, autotexts = ax.pie(
    grouped_data['count'], 
    labels=grouped_data['category'], 
    autopct='%1.1f%%', 
    startangle=140, 
    colors=colors,
    explode=explode,  # Выделяем кусочки
    #shadow=True,  # Добавляем тень для эффекта объема
    textprops={'fontsize': 11, 'color': 'black'}
)

# Добавляем легенду
ax.legend(
    wedges, 
    grouped_data['category'], 
    title="Категории зданий", 
    loc="center left", 
    bbox_to_anchor=(1, 0, 0.5, 1),  # Размещаем легенду справа от диаграммы
    fontsize=12
)

# Настройки заголовка
ax.set_title('Распределение зданий по категориям в районах велосипедной доставки', fontsize=16, pad=20)

# Показываем диаграмму
plt.tight_layout()  # Убедимся, что легенда не обрезается
plt.show()

In [ ]:
building_type_counts_bike = buildings_in_bike_isochrones.groupby(
    ['name', 'properties.local_name', 'building']
).size().reset_index(name='count')

# Преобразуем в таблицу
building_type_pivot_bike = building_type_counts_bike.pivot(
    index=['name', 'properties.local_name'], columns='building', values='count'
).fillna(0)

building_type_pivot_bike

In [ ]:
# Переименуем колонку 'yes' в 'дома'
table_bike= building_type_pivot_bike.rename(columns={'yes': 'дома'})

# Создаем новую колонку 'другие здания', суммируя все остальные колонки
table_bike['другие здания'] = table_bike.drop(columns=['дома']).sum(axis=1)

# Оставляем только нужные колонки
table_bike_simplified = table_bike[['дома', 'другие здания']]

table_bike_simplified

In [ ]:
# Преобразуем DataFrame в "длинный" формат
table_long = table_bike_simplified.reset_index().melt(
    id_vars=['name', 'properties.local_name'], 
    var_name='building_type', 
    value_name='count'
)

# Визуализация столбчатой диаграммы с накоплением
plt.figure(figsize=(14, 8))
sns.barplot(data=table_long, x='name', y='count', hue='building_type')
plt.title('Количество домов и других зданий для каждого ресторана и района')
plt.xlabel('Ресторан и район')
plt.ylabel('Количество зданий')
plt.legend(title='Тип здания', bbox_to_anchor=(1.05, 1), loc='upper left')
plt.xticks(rotation=45)
plt.show()

In [ ]:
map = folium.Map(location=center_point, zoom_start=11, attributionControl=0, tiles='cartodbpositron')

default_restaurant = "Lamvil"  

for _, restaurant in df_point.iterrows():
    restaurant_name = restaurant['name']
    restaurant_lat = restaurant['geometry'].y
    restaurant_lon = restaurant['geometry'].x

    show_layer = (restaurant_name == default_restaurant)

    restaurant_group = folium.FeatureGroup(name=restaurant_name, show=show_layer)

    isochron = isochrones_bike_gdf[isochrones_bike_gdf['name'] == restaurant_name].iloc[0]

    folium.GeoJson(
        isochron['geometry'],
        style_function=lambda x: {'color': 'orange', 'fillOpacity': 0.0, 'weight': 2},
    ).add_to(restaurant_group)

    filtered_buildings = buildings_in_bike_isochrones[
        buildings_in_bike_isochrones.intersects(isochron['geometry'])
    ]

    filtered_buildings = filtered_buildings[filtered_buildings.geometry.type == 'Polygon']

    if not filtered_buildings.empty:
        folium.GeoJson(
            filtered_buildings,
            style_function=lambda feature: {
                'fillColor': 'orange',
                'color': 'orange',
                'weight': 0.2,
                'fillOpacity': 0.6
            },
            highlight_function=lambda feature: {
                'fillColor': 'red',
                'color': 'red',
                'weight': 1,
                'fillOpacity': 0.9
            },
        ).add_to(restaurant_group)

    folium.Marker(
        [restaurant_lat, restaurant_lon],
        popup=restaurant_name,
        icon=folium.Icon(color="black", icon="cutlery")
    ).add_to(restaurant_group)

    restaurant_group.add_to(map)

folium.LayerControl(collapsed=False).add_to(map)

map

- автомобильные доставки

In [ ]:
filtered_buildings = filtered_osm_data.to_crs(isochrones_drive_gdf.crs)
# Пространственное соединение зданий и районов
filtered_buildings = gpd.sjoin(filtered_buildings, geo_data, how="inner", op="within")
filtered_buildings.reset_index(drop=True, inplace=True)

filtered_buildings

In [118]:
# Переименовываем колонки, если они есть
if 'index_left' in filtered_buildings.columns:
    filtered_buildings = filtered_buildings.rename(columns={'index_left': 'index_left_old'})
if 'index_right' in filtered_buildings.columns:
    filtered_buildings = filtered_buildings.rename(columns={'index_right': 'index_right_old'})

In [ ]:
buildings_in_drive_isochrones = gpd.sjoin(
    filtered_buildings, 
    isochrones_drive_gdf, 
    how="inner",  # Оставляем только здания, которые попадают в изохроны
    op="within"   # Здания должны находиться внутри изохрон
)

# Убираем дубликаты (если здание попадает в несколько изохрон)
buildings_in_drive_isochrones = buildings_in_drive_isochrones.drop_duplicates(subset=['geometry'])

# Удаляем лишние колонки (если нужно)
buildings_in_drive_isochrones = buildings_in_drive_isochrones.drop(columns=['index_right'])

# Сбрасываем индекс
buildings_in_drive_isochrones.reset_index(drop=True, inplace=True)

buildings_in_drive_isochrones

In [ ]:
building_drive_counts_df = buildings_in_drive_isochrones.groupby('building').size().reset_index(name='count')
building_drive_counts_df

In [ ]:
building_type_counts_drive = buildings_in_drive_isochrones.groupby(
    ['name', 'properties.local_name', 'building']
).size().reset_index(name='count')

# Преобразуем в таблицу
building_type_pivot_drive = building_type_counts_drive.pivot(
    index=['name', 'properties.local_name'], columns='building', values='count'
).fillna(0)

building_type_pivot_drive

In [ ]:
# Переименуем колонку 'yes' в 'дома'
table_drive = building_type_pivot_drive.rename(columns={'yes': 'дома'})

# Создаем новую колонку 'другие здания', суммируя все остальные колонки
table_drive['другие здания'] = table_drive.drop(columns=['дома']).sum(axis=1)

# Оставляем только нужные колонки
table_drive_simplified = table_drive[['дома', 'другие здания']]

table_drive_simplified

In [ ]:
counts_buildings_df = building_walk_counts_df.merge(building_bike_counts_df, on='building', suffixes=('_walk', '_bike'))
counts_buildings_df = counts_buildings_df.merge(building_drive_counts_df, on='building')

counts_buildings_df

Так как автомобильная доставка почти одинаковая во всех случаях отрисуем на карте просто все дома из датафрейма с домами автомообильной доставки

In [ ]:
center_point = [buildings_in_drive_isochrones.geometry.centroid.y.mean(), buildings_in_drive_isochrones.geometry.centroid.x.mean()]

map = folium.Map(location=center_point, zoom_start=11, attributionControl=False, tiles='cartodbpositron')

for _, row in buildings_in_drive_isochrones.iterrows():
    geom = row['geometry']
    if geom.geom_type in ['Polygon', 'MultiPolygon']:
        folium.GeoJson(
            geom,
            style_function=lambda feature: {
                'fillColor': 'purple',
                'color': 'purple',
                'weight': 0.5,
                'fillOpacity': 0.8
            }
        ).add_to(map)

map